In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import openpyxl
import selenium

In [2]:
import os
base = os.path.abspath('..')
print(base)


c:\Users\Shared Waters Lab\search-terms


In [3]:
# read excel (downloaded from Shared Waters Lab's shared Box drive)
events_path = os.path.join(base, 'Data', 'Events_2008.xls')
events_df = pd.read_excel(events_path)

In [4]:
# check out the basin data
spatial_path = os.path.join(base, 'Data', 'Basins_2024.csv')
spatial_df = pd.read_csv(spatial_path)

In [5]:
#events_df # needs a year column
events_df['DATE'] = pd.to_datetime(events_df['DATE'])
events_df['YEAR'] = events_df['DATE'].dt.year
events_df['YEAR'] = events_df['YEAR'].astype(pd.Int64Dtype())

In [6]:
events_yearfilter = events_df[events_df['YEAR']>1999]
#events_yearfilter.head()

## everything below here is filtered to be after 2000

In [7]:
# basin codes and their event counts

# Calculate value frequencies for the 'bcode' column
filtered_event_counts = events_yearfilter['BCode'].value_counts()
event_frequency_df = filtered_event_counts.to_frame('frequency').reset_index()

# Rename the columns
event_frequency_df.columns = ['BCODE', 'frequency']

print("BCode and Event Frequency Since 2000:")
print(event_frequency_df)

BCode and Event Frequency Since 2000:
   BCODE  frequency
0   MEKO        403
1   DANU        221
2   INDU        200
3   GANG        132
4   NILE         97
..   ...        ...
64  ORAL          1
65  TUMN          1
66  YALU          1
67  ORAN          1
68  IRWD          1

[69 rows x 2 columns]


In [17]:
post_2000s_events_basins = event_frequency_df['BCODE'].unique()

In [ ]:
low_freq_past_2000 = event_frequency_df[event_frequency_df['frequency'] < 5]
single_freq_past_2000 = event_frequency_df[event_frequency_df['frequency'] ==1]

# Add year column from events file
# Filter events for single frequency basins
single_events_for_year = events_yearfilter[events_yearfilter['BCode'].isin(single_freq_past_2000['BCODE'])].copy()
single_events_for_year.rename(columns={'BCode': 'BCODE'}, inplace=True)

# Get year for each basin (since frequency is 1, each basin has exactly one event)
year_mapping = single_events_for_year[['BCODE', 'YEAR']].drop_duplicates()

# Merge year into single_freq_past_2000
single_freq_past_2000 = pd.merge(single_freq_past_2000, year_mapping, on='BCODE', how='left')

#low_freq_past_2000

In [9]:
single_freq_past_2000

,BCODE,frequency
49,DRIN,1
50,SCRO,1
51,BANN,1
52,OYPK,1
53,AWSH,1
54,VRDR,1
55,ZARM,1
56,TUMB,1
57,GLOK,1
58,PUNG,1


In [13]:
single_event_basins_path = f'{base}/Data/single_event_basins.csv'
single_event_basins_path

'c:\\Users\\Shared Waters Lab\\search-terms/Data/single_event_basins.csv'

In [14]:
single_freq_past_2000.to_csv(single_event_basins_path)

In [15]:
old_terms_path = os.path.join(base, 'Data', 'Search_terms_2008.xlsx')
old_terms_df = pd.read_excel(old_terms_path)
new_terms_path = os.path.join(base, 'Data', 'Search_terms_2025.xlsx')
new_terms_df = pd.read_excel(new_terms_path)

In [18]:
# start by filtering the old terms df for the basins post-2000
old_terms_df = old_terms_df[old_terms_df['BCODE'].isin(post_2000s_events_basins)]
# rename the column to 'old_terms'
old_terms_df.rename(columns={'Basin_Specific_Terms': 'old_terms'}, inplace=True)
# filter the new terms df for the single frequency basins
new_terms_df = new_terms_df[new_terms_df['BCODE'].isin(post_2000s_events_basins)]
# rename the column to 'new_terms'
new_terms_df.rename(columns={'Basin_Specific_Terms': 'new_terms'}, inplace=True)
# join the two dataframes on the BCODE column
searchterms_df = pd.merge(old_terms_df, new_terms_df, on='BCODE', how='left')
searchterms_df = pd.merge(searchterms_df, event_frequency_df, on='BCODE')
keep_columns = ['BCODE', 'Basin_Name_x', 'frequency', 'old_terms', 'new_terms']
searchterms_df = searchterms_df[keep_columns]
searchterms_df.rename(columns={'Basin_Name_x':'Basin_Name'}, inplace=True)

searchterms_df.head()

,BCODE,Basin_Name,frequency,old_terms,new_terms
0,AMUR,Amur,14,Amur OR Emur OR Huma OR Muling OR Naoli OR Son...,"Amur OR Heilongjiang OR Arguny OR ""Erguna He"" ..."
1,AMZN,Amazon,10,Amazon OR Apere OR Baures OR Beni OR Guapore ...,Juruá OR Amazon OR Purus OR Putumayo OR Yavarí...
2,ARAL,Aral Sea,36,Aral OR Syr Daria OR Naryn OR Amu Daria OR Syr...,"""Aral Sea"" OR Panj OR Sir-darya OR Naryn OR ""A..."
3,ASIX,Asi/Orontes,2,Asi OR Orontes OR Amik Golu,"Orontes OR Asi OR ""Nahr al-Asi"" OR ""Asi Nehri""..."
4,ATRK,Atrak,1,Atrak OR Sumber,Atrak OR atruk OR Sumbar OR Atrek OR Çandyr OR...


In [24]:
single_frequency_searchterms = searchterms_df[searchterms_df['frequency']==1]

# Add year column from events file (filtered for year > 1999 to match frequency calculation)
# Filter events_yearfilter for single frequency basins
single_events_for_year = events_yearfilter[events_yearfilter['BCode'].isin(single_frequency_searchterms['BCODE'])].copy()
single_events_for_year.rename(columns={'BCode': 'BCODE'}, inplace=True)

# Get year for each basin (since frequency is 1, each basin has exactly one event after 1999)
year_mapping = single_events_for_year[['BCODE', 'YEAR']].drop_duplicates()

# Merge year into single_frequency_searchterms
single_frequency_searchterms = pd.merge(single_frequency_searchterms, year_mapping, on='BCODE', how='left')

#check that it's only one frequency per bcode now
#single_frequency_searchterms #?

#and if that's a yes, drop column frequency 
single_frequency_searchterms = single_frequency_searchterms.drop('frequency', axis=1)
single_frequency_searchterms.head()
# single_basins = single_frequency_searchterms['BCODE'].unique()
# len(single_basins)

,BCODE,Basin_Name,old_terms,new_terms,YEAR
0,ATRK,Atrak,Atrak OR Sumber,Atrak OR atruk OR Sumbar OR Atrek OR Çandyr OR...,2004
1,AWSH,Awash,Abhe Bid Hayak OR Awash OR Finochaa OR Gemeri ...,"Awash OR Germama OR Kesem OR ""Jirjir Shet'"" OR...",2002
2,BANN,Bann,Bann OR CAPS(Black Water) OR Laugh Neagh OR CA...,"Bann OR Blackwater OR ""Lower Bann"" OR ""Upper B...",2002
3,DRIN,Drin,Cemerice OR Drin OR Drinizi OR Fshat OR Gjemon...,"Drin OR ""Drini i Bardhë"" OR Morača OR ""Drini i...",2001
4,DURO,Douro/Duero,Adaja OR Aliste OR Bembezar OR Bibey OR Cea OR...,Douro OR Duero OR Pisuerga OR Esla OR Tormes O...,2005


In [25]:
single_frequency_searchterms.to_csv(single_event_basins_path)

In [ ]:
# low_frequency_df = event_frequency_df[event_frequency_df['frequency'] < 5]
# low_frequency_bcode_list = low_frequency_df['BCode'].tolist()
# len(low_frequency_bcode_list)

In [ ]:
# would also be interested in finding the date for each basin's single event
# Filter events for single frequency basins and create a copy to avoid warnings
single_events_filter = events_df[events_df['BCode'].isin(single_basins)].copy()
single_events_filter.rename(columns={'BCode': 'BCODE'}, inplace=True)

# Extract year from DATE column (already done in events_df, but ensuring it's there)
single_events_filter['YEAR'] = pd.to_datetime(single_events_filter['DATE'], errors='coerce').dt.year

# Create old_year_summary with just BCODE and YEAR columns
old_year_summary = single_events_filter[['BCODE', 'YEAR']].drop_duplicates().copy()

# Display the summary
old_year_summary.head()

## ok 

for setup
- I want the dataframe to be *all* basins with columns ['BCODE', 'Basin_Name', 'events_frequency', 'old_terms', 'new_terms', 'old_terms_resultcount', 'new_terms_resultcount']
- I want streamlit app to just queue em all. I want it to do this from a list.
- I actually want a first round of this to just get the result count for each and put it in a dataframe, which means...


I want this script to:
- download word docs instead, so in a test round I want it to pause on the dialog window or even just the results page, just to literally stop there so I can figure out the right selectors, because...
- I want to just download the resultslist. so getresultcount will break it up by thousands.
- download to Data/Downloads (will have to update camelcase in paths)


In [7]:
single_frequency_df = event_frequency_df[event_frequency_df['frequency'] == 1]
single_frequency_bcode_list = single_frequency_df['BCode'].tolist()
len(single_frequency_bcode_list)

31

In [ ]:
# ideally end up with additional columns 'old_terms_resultcount', 'new_terms_resultcount'

,BCODE,YEAR
922,MDJD,1964
1612,SEPK,1973
1613,FLYX,1973
1891,CRTY,1978
2024,AKPA,1982


In [23]:
# merge the year
scraper_df = pd.merge(single_frequency_searchterms, old_year_summary, on='BCODE')
scraper_df

,BCODE,Basin_Name,old_terms,new_terms,YEAR
0,AKPA,Akpa,Akpa,Akpa OR Akpakorum OR Ebe OR Ikpan OR Akpasang ...,1982
1,BANN,Bann,Bann OR CAPS(Black Water) OR Laugh Neagh OR CA...,"Bann OR Blackwater OR ""Lower Bann"" OR ""Upper B...",2002
2,BRTA,Barta,Barta,Barta OR Bārta OR Vārtāja OR Ālande OR Luoba O...,1997
3,BUZI,Buzi,Buzi OR Rio Revue OR Chibudzana OR Nyanhombo O...,Buzi OR Búzi OR Lucite OR Revué OR Mossurize O...,1995
4,CRTY,Corantijn/Courantyne,Corantijn OR Courantyne,Courantyne OR Corantijn OR Curuni OR Upper-Cor...,1978
5,CTTB,Catatumbo,Catatumbo,"Catatumbo OR ""de Oro"" OR Zulia OR Tarra OR Alg...",1993
6,ELNK,Elancik,Elancik,"Elancik OR ""Sukhoy Elanchik"" OR ""Mokryy Elanch...",1997
7,ETOS,Cuvelai/Etosha,Cuvelai OR Etosha,"Cuvelai OR Etosha OR ""Omuramba Ovambo"" OR Ekum...",1995
8,FLYX,Fly,CAPS(Fly),"Fly OR ""Ok Tedi"" OR ""Ok Birim"" OR Strickland O...",1973
9,GOSR,Goascoran,Goascoran,"Goascoran OR Goascorán OR ""San Juan"" OR Amatil...",1995


In [25]:
y2k_scraper = scraper_df[scraper_df['YEAR']>1999]
y2k_scraper

,BCODE,Basin_Name,old_terms,new_terms,YEAR
1,BANN,Bann,Bann OR CAPS(Black Water) OR Laugh Neagh OR CA...,"Bann OR Blackwater OR ""Lower Bann"" OR ""Upper B...",2002
16,OYPK,Oiapoque/Oyupock,Oiapoque OR Oyupock OR Camopi,Oiapoque OR Oyupock OR Camopi OR Anotaie OR Cr...,2002
18,PUNG,Pungwe,Pungwe,Pungwe OR Púnguè OR Vunduzi OR Nhandugue OR Ur...,2002
22,SCRO,St. Croix,St. Croix OR Chiputneticook,"""St. Croix"" OR ""Saint Croix"" OR Monument OR ""E...",2001


what needs to change in the mini-scraper?

* date range will not be hard-coded, will take in data from above df
* - or actually, it could be hard-coded as just a smaller range (2000-2008 or so)
* column 'start_date' and 'end_date', and it'll be f('01-01-{YEAR}') and f('12-31-{YEAR}') in date range field
* maybe don't run in app.py, make it a terminal process if possible
* this could also be a much simpler process, downloading just the metadata/resultslist

L M A O 
I just realized nobody can do a demo because OSU doesn't have a nexis uni license anymore